### Article Views 1

In [33]:
import pandas as pd
import pandasql as ps

data = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
Views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})
print(Views)

   article_id  author_id  viewer_id  view_date
0           1          3          5 2019-08-01
1           1          3          6 2019-08-02
2           2          7          7 2019-08-01
3           2          7          6 2019-08-02
4           4          7          1 2019-07-22
5           3          4          4 2019-07-21
6           3          4          4 2019-07-21


In [47]:
print(pd.DataFrame(Views[Views['author_id'] == Views['viewer_id']]['author_id']).drop_duplicates().rename(columns={'author_id':'id'}).sort_values('id').reset_index(drop=True))

   id
0   4
1   7


In [46]:
query = '''
select distinct(author_id) as id
from Views
where author_id = viewer_id
order by id
'''

print(ps.sqldf(query, locals()))

   id
0   4
1   7


### Invalid Tweets

In [48]:
data = [[1, 'Vote for Biden'], [2, 'Let us make America great again!']]
Tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})
print(Tweets)

   tweet_id                           content
0         1                    Vote for Biden
1         2  Let us make America great again!


In [83]:
Tweets[Tweets['content'].str.len()>15][['tweet_id']]

,tweet_id
1,2


In [86]:
query = '''
select tweet_id
from Tweets
where length(content) > 15
'''

print(ps.sqldf(query, locals()))

   tweet_id
0         2


### Calculate Special Bonus

In [137]:
data = [[2, 'Meir', 3000], [3, 'Michael', 3800], [7, 'Addilyn', 7400], [8, 'Juan', 6100], [9, 'Kannon', 7700]]
Employees = pd.DataFrame(data, columns=['employee_id', 'name', 'salary']).astype({'employee_id':'int64', 'name':'object', 'salary':'int64'})
print(Employees)

   employee_id     name  salary
0            2     Meir    3000
1            3  Michael    3800
2            7  Addilyn    7400
3            8     Juan    6100
4            9   Kannon    7700


In [88]:
Employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_id  5 non-null      int64 
 1   name         5 non-null      object
 2   salary       5 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 252.0+ bytes


In [145]:
Employees.loc[(Employees['employee_id']%2 == 1)&(~Employees['name'].str.startswith('M')), 'bonus'] = Employees['salary']
Employees['bonus'] = Employees['bonus'].fillna(0)
print(Employees[['employee_id','bonus']])

   employee_id   bonus
0            2     0.0
1            3     0.0
2            7  7400.0
3            8     0.0
4            9  7700.0


In [143]:
query = '''
select employee_id, 
if(employee_id % 2 != 0 and left(name,1) <> 'M', salary, 0 ) as bonus
from Employees
order by employee_id
'''

other_query = '''
SELECT employee_id, 
CASE WHEN employee_id % 2 != 0 AND SUBSTR(name, 1, 1) <> 'M' THEN salary ELSE 0 END AS bonus
FROM Employees
ORDER BY employee_id
'''

print(ps.sqldf(other_query , locals()))

   employee_id  bonus
0            2      0
1            3      0
2            7   7400
3            8      0
4            9   7700
